In [1]:
import re
import requests
from lxml import etree
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
from threading import Thread

In [2]:
num = 1
for page in range(10, 13): # A simple way to download many pages.
    url = "https://kissgoddess.com/album/30485_%d.html" % page
    req = requests.get(url)
    req.encoding = 'utf-8'
    html = req.text
    # print(req.status_code)
    # print(html)

    tree = etree.HTML(html)
#     print(tree)
    result = tree.xpath('//div[@class="td-gallery-content"]/img/@src')
#     print(result)
    for link in result:
        print("download: ", link)
        urlretrieve(link, "E:\crawler_pictures\practice1\%d.png"%num)
        num += 1

In [3]:
url = "https://kissgoddess.com/album/30485.html"
url2 = "https://kissgoddess.com"
num = 1
for n in range(3):
    req = requests.get(url)
    req.encoding = 'utf-8'
    html = req.text
#     print(req.status_code)
#     print(html, '\n'*2)
    tree = etree.HTML(html)
    result = tree.xpath('//div[@class="td-gallery-content"]/img/@src')
#     print(result)
    for link in result:
        print("download: ", link)
        urlretrieve(link, "E:\crawler_pictures\practice1\%d.png"%num)
        num += 1
    current_page = tree.xpath('//a[@class="a1"]/@href')
    url = url2 + current_page[1] # A better way to download many pages.

In [4]:
# Use BeautifulSoup !!!
url = "https://kissgoddess.com/album/28637_2.html"
url2 = "https://kissgoddess.com"
num = 1
for i in range(2):
    req = requests.get(url)
    req.encoding = 'utf-8'
    html = req.text
    soup = BeautifulSoup(html)
    result = soup.find_all('div', class_="td-gallery-content")
    
    # line 17-21 are very important
    # the type of result is <class 'bs4.element.ResultSet'> 
    # the type of result[0] is <class 'bs4.element.Tag'> 
    # only "Tag" can use ".descendants" to find next tag.
    # sometimes the child of result includes '\n', we must delete them.
    child = [] 
    for child1 in result[0].descendants:  
        child.append(child1)
    while '\n' in child:
        child.remove('\n')
    retries = 0    
    for i in child:
        link = i.get('src')
        print("download: ", link)
        while retries < 3: # if failed try again, until tried three times
#             try:
#                 urlretrieve(link, "E:\crawler_pictures\practice1\pic%d.png"%num)
#                 num += 1
            # A better way to save:
            try:
                pic = requests.get(link, timeout = 30) # if wait too long, give up
                with open("E:\crawler_pictures\practice1\pic%d.png"%num, 'wb') as f:
                    f.write(pic.content) 
                    # Pay attention! binary files should use "content" !!!
                num += 1
            except:
                print("Download failed!")
                retries += 1
            else:
                print("saved!!!")
                break
    page_info = soup.find_all('a', class_='a1')[1]
    url = url2 + page_info.get('href')

In [5]:
# Use multithreading!!!
url = "https://kissgoddess.com/album/27471.html"
url2 = "https://kissgoddess.com"
def download(link):
    retries = 0
    filename = link.split('/')[-1]
    while retries < 3:
        try:
            pic = requests.get(link, timeout=40)
            with open("E:\crawler_pictures\practice1\%s"%filename, 'wb') as f:
                f.write(pic.content)
        except:
            print("Download failed!!!")
            retries += 1
        else:
            print("saved!!!")
            break
                       
for n in range(10):
    req = requests.get(url)
    req.encoding = 'utf-8'
    html = req.text
    tree = etree.HTML(html)
    result = tree.xpath('//div[@class="td-gallery-content"]/img/@src')
    for link in result:
        print("download: ", link)
        t = Thread(target=download, args=(link,))
        t.start()
    current_page = tree.xpath('//a[@class="a1"]/@href')
    url = url2 + current_page[1] # A better way to download many pages.

In [7]:
# Use functions
def get_pic_links(url):
    req = requests.get(url)
    req.encoding = 'utf-8'
    html = req.text
    tree = etree.HTML(html)
    result = tree.xpath('//div[@class="td-gallery-content"]/img/@src')
    links = []
    for link in result:
        links.append(link)
    return links


def download(link):
    retries = 0
    filename = link.split('/')[-1]
    while retries < 3:
        try:
            pic = requests.get(link, timeout=40)
            with open("E:\crawler_pictures\practice1\%s"%filename, 'wb') as f:
                f.write(pic.content)
        except:
            print("Download failed!!!")
            retries += 1
        else:
            print("saved!!!")
            break
            

def use_multithreading(url):
    pic_links = get_pic_links(url)
    threads = []  
    for link in pic_links:
        print("download: ", link)
        t = Thread(target=download, args=(link,))
        threads.append(t)
    for i in threads:
        i.start()

        
def update_page(url):
    req = requests.get(url)
    req.encoding = 'utf-8'
    html = req.text
    tree = etree.HTML(html)
    current_page = tree.xpath('//a[@class="a1"]/@href')
    url2 = url.rsplit('/', 2)[0] # use "rsplit" to split from back to front !!!
    new_url = url2 + current_page[1]
    return new_url


def main(init_url, pages):
    if pages < 1:
        return "end"
    else:
        use_multithreading(init_url)
        new_url = update_page(init_url)
        return main(new_url, pages-1)
    
init_url = "https://kissgoddess.com/album/28637_1.html"
main(init_url, 6)

download:  https://pic.kissgoddess.com/gallery/26454/28637/s/0.jpg
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/001.jpg
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/002.jpg
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/003.jpg
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/004.jpg
saved!!!
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/005.jpg
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/006.jpg
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/007.jpg
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/008.jpg
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/009.jpg
saved!!!
saved!!!
saved!!!
saved!!!
saved!!!
saved!!!
saved!!!
saved!!!
saved!!!
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/010.jpg
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/011.jpg
download:  https://pic.kissgoddess.com/gallery/26454/28637/s/012.jpg
download:  http

'end'

saved!!!
saved!!!
saved!!!
saved!!!
saved!!!
saved!!!
saved!!!
saved!!!
saved!!!
saved!!!
